In [1]:
import os
import csv
import time
import urllib3
from datetime import datetime
from trino.dbapi import connect
from trino.auth import BasicAuthentication  # Import for LDAP authentication

# Disable SSL verification warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Trino Connection Details
TRINO_HOST = "192.168.80.155"
TRINO_PORT = "30071"
TRINO_USER = "ctzn.bank"
TRINO_PASSWORD = "ctzn.bank_123"
TRINO_CATALOG = "iceberg"
TRINO_SCHEMA = "silver_crmuser"
TRINO_HTTP_SCHEME = "https"  # Secure connection

# Output CSV filename
OUTPUT_FILE = "ctzn_5_years_data.csv"

# SQL Query
SQL_QUERY = """ 
WITH recent_customers AS (
    SELECT DISTINCT 
        g.cif_id
    FROM 
        gold.dim_gam AS g
    WHERE 
        CAST(DATE_PARSE(SUBSTRING(g.acct_opn_date, 1, 19), '%Y-%m-%dT%H:%i:%s') AS DATE) >= CURRENT_DATE - INTERVAL '5' YEAR
),

account_activity AS (
    SELECT 
        a.cif_id,
        -- Balance calculations
        SUM(COALESCE(a.total_credit_tran_vol, 0) - COALESCE(a.total_debit_tran_vol, 0)) AS balance,
        COUNT(DISTINCT a.nepali_month)/6.0 AS balance_frequency,
        
        -- Purchase metrics
        SUM(COALESCE(a.total_debit_tran_vol, 0)) AS purchases,
        MAX(COALESCE(a.total_debit_tran_vol, 0)) AS oneoff_purchases,
        SUM(COALESCE(a.total_debit_tran_vol, 0)) - MAX(COALESCE(a.total_debit_tran_vol, 0)) AS installments_purchases,
        
        -- Cash advance calculations
        SUM(CASE WHEN COALESCE(a.total_credit_tran_vol, 0) > 0 AND COALESCE(a.total_debit_tran_vol, 0) = 0 
                 THEN COALESCE(a.total_credit_tran_vol, 0) ELSE 0 END) AS cash_advance,
        
        -- Frequency calculations
        COUNT(DISTINCT a.foracid)/6.0 AS purchases_frequency,
        COUNT(DISTINCT CASE WHEN COALESCE(a.total_debit_tran_vol, 0) > 100000 THEN a.foracid END)/6.0 AS oneoff_purchases_frequency,
        COUNT(DISTINCT CASE WHEN COALESCE(a.total_debit_tran_vol, 0) BETWEEN 1 AND 100000 THEN a.foracid END)/6.0 AS purchases_installments_frequency,
        COUNT(DISTINCT CASE WHEN COALESCE(a.total_credit_tran_vol, 0) > 0 THEN a.foracid END)/6.0 AS cash_advance_frequency,
        
        -- Transaction counts
        COUNT(DISTINCT CASE WHEN COALESCE(a.total_credit_tran_vol, 0) > 0 THEN a.foracid END) AS cash_advance_trx,
        COUNT(DISTINCT a.foracid) AS purchases_trx,
        
        -- Payment behavior
        SUM(COALESCE(a.total_credit_tran_vol, 0)) AS payments,
        COUNT(DISTINCT CASE WHEN COALESCE(a.total_credit_tran_vol, 0) >= COALESCE(a.total_debit_tran_vol, 0) 
                            THEN a.nepali_month END)/6.0 AS prc_full_payment
    FROM 
        gold.mv_fact_deposit_account_insights a
    JOIN 
        recent_customers rc ON a.cif_id = rc.cif_id
    GROUP BY 
        a.cif_id
),

salary_stats AS (
    SELECT 
        APPROX_PERCENTILE(COALESCE(salary_per_month, 0), 0.5) AS median_salary,
        APPROX_PERCENTILE(COALESCE(salary_per_month, 0), 0.05) AS fifth_percentile_salary
    FROM 
        gold.dim_customers
),

customer_profile AS (
    SELECT 
        g.cif_id,
        -- Tenure calculation from account opening date
        DATE_DIFF('year', 
                 CAST(DATE_PARSE(SUBSTRING(MIN(g.acct_opn_date), 1, 19), '%Y-%m-%dT%H:%i:%s') AS DATE), 
                 CURRENT_DATE) AS tenure,
        
        -- Estimated minimum payments (5% of median salary)
        (SELECT fifth_percentile_salary FROM salary_stats) AS minimum_payments
    FROM 
        gold.dim_gam g
    LEFT JOIN 
        gold.dim_customers c ON g.cif_id = c.cif_id
    GROUP BY 
        g.cif_id
)

SELECT 
    aa.cif_id AS custid,
    aa.balance,
    aa.balance_frequency,
    aa.purchases,
    aa.oneoff_purchases,
    aa.installments_purchases,
    aa.cash_advance,
    aa.purchases_frequency,
    aa.oneoff_purchases_frequency,
    aa.purchases_installments_frequency,
    aa.cash_advance_frequency,
    aa.cash_advance_trx,
    aa.purchases_trx,
    -- Using estimated credit limit (3x median salary)
    (SELECT median_salary * 3 FROM salary_stats) AS credit_limit,
    aa.payments,
    cp.minimum_payments,
    aa.prc_full_payment,
    cp.tenure
FROM 
    account_activity aa
JOIN 
    customer_profile cp ON aa.cif_id = cp.cif_id
ORDER BY 
    aa.cif_id;
"""

def fetch_data():
    """Fetches data from Trino and saves it locally as a CSV file."""
    
    try:
        print("Connecting to Trino with LDAP authentication...")
        conn = connect(
            host=TRINO_HOST,
            port=TRINO_PORT,
            user=TRINO_USER,
            auth=BasicAuthentication(TRINO_USER, TRINO_PASSWORD),
            catalog=TRINO_CATALOG,
            schema=TRINO_SCHEMA,
            http_scheme=TRINO_HTTP_SCHEME,  # Use HTTPS
            request_timeout=600,  # Increase timeout to 10 minutes
            verify=False  # <--- This disables SSL verification
        )
        cursor = conn.cursor()

        print("Executing query...")
        cursor.execute(SQL_QUERY)

        # Fetch column headers
        columns = [desc[0] for desc in cursor.description]

        # Save data to CSV file locally
        with open(OUTPUT_FILE, "w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(columns)  # Write headers
            
            # Fetch rows in streaming mode
            while True:
                rows = cursor.fetchmany(1000)  # Fetch 1000 rows at a time
                if not rows:
                    break
                writer.writerows(rows)

        # Close connection
        conn.close()
        print(f"Data successfully saved to {OUTPUT_FILE}")
        return OUTPUT_FILE  

    except Exception as e:
        print(f"Error: {str(e)}")
        return None

# Run the function
csv_file_path = fetch_data()
print(f"CSV file is saved at: {csv_file_path}")


Connecting to Trino with LDAP authentication...
Executing query...
Error: TrinoUserError(type=USER_ERROR, name=SYNTAX_ERROR, message="line 100:14: mismatched input ';'. Expecting: '%', '*', '+', ',', '-', '.', '/', 'AND', 'ASC', 'AT', 'DESC', 'FETCH', 'LIMIT', 'NULLS', 'OFFSET', 'OR', '[', '||', <EOF>, <predicate>", query_id=20250407_105809_00522_ir27g)
CSV file is saved at: None


In [2]:
pip install trino

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 787.3 kB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.1 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import csv
import time
import urllib3
from datetime import datetime
from trino.dbapi import connect
from trino.auth import BasicAuthentication

# Disable SSL verification warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Trino Connection Details
TRINO_HOST = "192.168.80.155"
TRINO_PORT = "30071"
TRINO_USER = "ctzn.bank"
TRINO_PASSWORD = "ctzn.bank_123"
TRINO_CATALOG = "iceberg"
TRINO_SCHEMA = "silver_crmuser"
TRINO_HTTP_SCHEME = "https"

# Output CSV filename
OUTPUT_FILE = "ctzn_10_years_data.csv"

# SQL Query - Fixed syntax and formatting
SQL_QUERY = """
WITH recent_customers AS (
    SELECT DISTINCT 
        g.cif_id
    FROM 
        gold.dim_gam AS g
    WHERE 
        CAST(DATE_PARSE(SUBSTRING(g.acct_opn_date, 1, 19), '%Y-%m-%dT%H:%i:%s') AS DATE) >= CURRENT_DATE - INTERVAL '10' YEAR
),

account_activity AS (
    SELECT 
        a.cif_id,
        -- Balance calculations
        SUM(COALESCE(a.total_credit_tran_vol, 0) - COALESCE(a.total_debit_tran_vol, 0)) AS balance,
        COUNT(DISTINCT a.nepali_month)/60.0 AS balance_frequency,
        
        -- Purchase metrics
        SUM(COALESCE(a.total_debit_tran_vol, 0)) AS purchases,
        MAX(COALESCE(a.total_debit_tran_vol, 0)) AS oneoff_purchases,
        SUM(COALESCE(a.total_debit_tran_vol, 0)) - MAX(COALESCE(a.total_debit_tran_vol, 0)) AS installments_purchases,
        
        -- Cash advance calculations
        SUM(CASE WHEN COALESCE(a.total_credit_tran_vol, 0) > 0 AND COALESCE(a.total_debit_tran_vol, 0) = 0 
                THEN COALESCE(a.total_credit_tran_vol, 0) ELSE 0 END) AS cash_advance,
        
        -- Frequency calculations
        COUNT(DISTINCT a.foracid)/60.0 AS purchases_frequency,
        COUNT(DISTINCT CASE WHEN COALESCE(a.total_debit_tran_vol, 0) > 100000 THEN a.foracid END)/60.0 AS oneoff_purchases_frequency,
        COUNT(DISTINCT CASE WHEN COALESCE(a.total_debit_tran_vol, 0) BETWEEN 1 AND 100000 THEN a.foracid END)/60.0 AS purchases_installments_frequency,
        COUNT(DISTINCT CASE WHEN COALESCE(a.total_credit_tran_vol, 0) > 0 THEN a.foracid END)/60.0 AS cash_advance_frequency,
        
        -- Transaction counts
        COUNT(DISTINCT CASE WHEN COALESCE(a.total_credit_tran_vol, 0) > 0 THEN a.foracid END) AS cash_advance_trx,
        COUNT(DISTINCT a.foracid) AS purchases_trx,
        
        -- Payment behavior
        SUM(COALESCE(a.total_credit_tran_vol, 0)) AS payments,
        COUNT(DISTINCT CASE WHEN COALESCE(a.total_credit_tran_vol, 0) >= COALESCE(a.total_debit_tran_vol, 0) 
                            THEN a.nepali_month END)/60.0 AS prc_full_payment
    FROM 
        gold.mv_fact_deposit_account_insights a
    JOIN 
        recent_customers rc ON a.cif_id = rc.cif_id
    GROUP BY 
        a.cif_id
),

salary_stats AS (
    SELECT 
        APPROX_PERCENTILE(COALESCE(salary_per_month, 0), 0.5) AS median_salary,
        APPROX_PERCENTILE(COALESCE(salary_per_month, 0), 0.05) AS fifth_percentile_salary
    FROM 
        gold.dim_customers
),

customer_profile AS (
    SELECT 
        g.cif_id,
        -- Tenure calculation from account opening date
        DATE_DIFF('year', 
                CAST(DATE_PARSE(SUBSTRING(MIN(g.acct_opn_date), 1, 19), '%Y-%m-%dT%H:%i:%s') AS DATE), 
                CURRENT_DATE) AS tenure,
        
        -- Estimated minimum payments (5% of median salary)
        (SELECT fifth_percentile_salary FROM salary_stats) AS minimum_payments
    FROM 
        gold.dim_gam g
    LEFT JOIN 
        gold.dim_customers c ON g.cif_id = c.cif_id
    GROUP BY 
        g.cif_id
)

SELECT 
    aa.cif_id AS custid,
    aa.balance,
    aa.balance_frequency,
    aa.purchases,
    aa.oneoff_purchases,
    aa.installments_purchases,
    aa.cash_advance,
    aa.purchases_frequency,
    aa.oneoff_purchases_frequency,
    aa.purchases_installments_frequency,
    aa.cash_advance_frequency,
    aa.cash_advance_trx,
    aa.purchases_trx,
    -- Using estimated credit limit (3x median salary)
    (SELECT median_salary * 3 FROM salary_stats) AS credit_limit,
    aa.payments,
    cp.minimum_payments,
    aa.prc_full_payment,
    cp.tenure
FROM 
    account_activity aa
JOIN 
    customer_profile cp ON aa.cif_id = cp.cif_id
ORDER BY 
    aa.cif_id
"""

def fetch_data():
    """Fetches data from Trino and saves it locally as a CSV file."""
    
    try:
        print("Connecting to Trino with LDAP authentication...")
        conn = connect(
            host=TRINO_HOST,
            port=TRINO_PORT,
            user=TRINO_USER,
            auth=BasicAuthentication(TRINO_USER, TRINO_PASSWORD),
            catalog=TRINO_CATALOG,
            schema=TRINO_SCHEMA,
            http_scheme=TRINO_HTTP_SCHEME,
            request_timeout=600,
            verify=False
        )
        cursor = conn.cursor()

        print("Executing query...")
        start_time = time.time()
        cursor.execute(SQL_QUERY)

        # Fetch column headers
        columns = [desc[0] for desc in cursor.description]

        # Save data to CSV file locally
        with open(OUTPUT_FILE, "w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(columns)  # Write headers
            
            # Fetch rows in batches
            batch_count = 0
            total_rows = 0
            while True:
                rows = cursor.fetchmany(5000)  # Fetch 5000 rows at a time
                if not rows:
                    break
                writer.writerows(rows)
                batch_count += 1
                total_rows += len(rows)
                # print(f"Fetched batch {batch_count} with {len(rows)} rows (total: {total_rows})")

        conn.close()
        elapsed = time.time() - start_time
        # print(f"Success! Saved {total_rows} rows to {OUTPUT_FILE} in {elapsed:.2f} seconds")
        return OUTPUT_FILE

    except Exception as e:
        print(f"Error occurred: {str(e)}")
        if 'conn' in locals():
            conn.close()
        return None

# Run the function
if __name__ == "__main__":
    csv_file_path = fetch_data()
    if csv_file_path:
        print(f"Data extraction complete. File saved at: {os.path.abspath(csv_file_path)}")
    else:
        print("Data extraction failed.")

Connecting to Trino with LDAP authentication...
Executing query...
Data extraction complete. File saved at: /home/jovyan/ctzn_data/customer-segmentation/ctzn_10_years_data.csv
